In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/clean_data.csv")
df = df.iloc[:,[2,4]]
df.head()

,label,clean_tweet
0,0,father dysfunct selfish drag kid dysfunction. ...
1,0,thank #lyft credit can't use caus offer wheelc...
2,0,bihday majesti
3,0,#model love u take u time urd+-!!! dddd d|d|d|
4,0,factsguide: societi #motiv


In [ ]:
df.dropna(subset = ["clean_tweet"], inplace=True)

In [ ]:
def remover(my_string = ""):
  values = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ012345678 ")
  for item in my_string:
    if item not in values:
      my_string = my_string.replace(item, "")
  return my_string
df.loc[:,'clean'] = [remover(x) for x in df.clean_tweet]
df = df.iloc[:,[0,2]]
dff = df.drop(['label'], axis = 1)
X_temp, X_test, y_temp, y_test = train_test_split(dff, list(df.label), test_size=0.1)

In [ ]:
X=X_temp
X['label'] = y_temp
nonhate = X[X['label']== 0]
hate = X[X.label == 1]
nonhatesample = nonhate.sample(n = hate.shape[0])
ds = pd.concat([hate, nonhatesample], axis = 0)
ds.to_csv("/content/dataset_train.csv")
ds_temp = ds
testdf = X_test
testdf['label'] = y_test
ds = pd.concat([ds_temp, testdf], axis = 0)

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(ds.clean,ds.label, test_size=0.1)
X_temp.head()

4594     whitenationalist leader reveal 5 horrifi hope ...
6878     abbi nike black write america pornhub click wa...
8601     see told you lot christma ball scatter around ...
3355             yep  grate everyday  gratitud laurasworld
23061    due natur work miss fav themorningfix catchup ...
Name: clean, dtype: object

In [ ]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from google.colab import drive
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
drive.mount('/content/gdrive')
path=f'/content/sentiment_analysis.pt'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Mounted at /content/gdrive


In [ ]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")

GPU is available


In [ ]:
def preprocess(s):
    s = re.sub(r"[^\w\s]", '', s)
    s = re.sub(r"\s+", '', s)
    s = re.sub(r"\d", '', s)
    
    return s

In [ ]:
def tockenize(x_train,x_val,x_test):
    word_list = []
    stop_words = set(stopwords.words('english')) 
    for sentence in x_train:
        for word in sentence.lower().split():
            word = preprocess(word)
            
            if word not in stop_words and word != '':
                lemmatizer = WordNetLemmatizer()
                wordnet_lemmatizer=lemmatizer.lemmatize(word)

                word_list.append(wordnet_lemmatizer)

    Count = Counter(word_list)
    
    corpus = sorted(Count,key=Count.get,reverse=True)

    freq_words = {w:i+1 for i,w in enumerate(corpus)}
    
    train_set,test_set,valid_set = [],[],[]
    for i in x_train:
            train_set.append([freq_words[preprocess(word)] for word in i.lower().split() 
                                     if preprocess(word) in freq_words.keys()])
    for i in x_val:
            valid_set.append([freq_words[preprocess(word)] for word in i.lower().split() 
                                    if preprocess(word) in freq_words.keys()])
    
    for i in x_test:
            test_set.append([freq_words[preprocess(word)] for word in i.lower().split()
                                  if preprocess(word) in freq_words.keys()])
    
    return np.array(train_set),np.array(test_set),np.array(valid_set),freq_words


In [ ]:
x_train,x_val,x_test,vocab = tockenize(X_temp,X_test,X_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
print(x_train)
x_train.shape

[list([2256, 706, 767, 1774, 133, 94])
 list([4587, 3008, 19, 604, 94, 4588, 1497, 81])
 list([20, 558, 243, 443, 848, 3009, 179, 4589]) ...
 list([1127, 13181, 197, 1576, 1127, 1576, 2839, 1965, 750, 3456])
 list([53, 263, 178, 9004, 840, 713, 37])
 list([42, 175, 1259, 550, 24, 456])]


(6498,)

In [ ]:
def padding(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [ ]:
x_train_pad = padding(x_train,500)
x_valid_pad = padding(x_test,500)


In [ ]:
x_train.shape

(6498,)

In [ ]:
train_y=np.array(y_temp)
valid_y=np.array(y_test)

In [ ]:
x_val.shape

(722,)

In [ ]:
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(x_valid_pad), torch.from_numpy(valid_y))
# training and validation dataloaders
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [ ]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

In [ ]:
class LSTM(nn.Module):
    def __init__(self,no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5):
        super(LSTM,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
 
        self.no_layers = no_layers
        self.vocab_size = vocab_size
    
        # embedding 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        
        # dropout layer/ regularization
        self.dropout = nn.Dropout(0.4)
    
        # linear and sigmoid layer
        self.lin = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self,x,hidden):
      
        batch_size = x.size(0)
        
        embeds = self.embedding(x)  
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
        
        out = self.dropout(lstm_out)
        out = self.lin(out)
        sig_out = self.sig(out)


        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1]
 
        return sig_out, hidden
    def init_hidden(self, batch_size):
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

    

In [ ]:
no_layers = 2
vocab_size = len(vocab) + 1 
embedding_dim = 64
output_dim = 1
hidden_dim = 128
model = LSTM(no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5)
model.to(device)

print(model)

LSTM(
  (embedding): Embedding(13182, 64)
  (lstm): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (lin): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
lr=0.001

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [ ]:
def training(batch_size,train_loader,valid_loader,):
  clip = 5
  epochs = 20
  valid_loss_min = np.Inf

  epoch_tr_loss,epoch_vl_loss = [],[]
  epoch_tr_acc,epoch_vl_acc = [],[]

  for epoch in range(epochs):
      train_losses = []
      train_acc = 0.0
      model.train()
 
      
      h = model.init_hidden(batch_size)
  
      for inputs, labels in train_loader:
          
          inputs, labels = inputs.to(device), labels.to(device)   
          h = tuple([each.data for each in h])
          
          model.zero_grad()
          output,h = model(inputs,h)
          loss = criterion(output.squeeze(), labels.float())
          loss.backward()
          train_losses.append(loss.item())
          accuracy = acc(output,labels)
          train_acc += accuracy
          
         
          nn.utils.clip_grad_norm_(model.parameters(), clip)
          optimizer.step()
  
      
        
      val_h = model.init_hidden(batch_size)
      val_losses = []

      val_acc = 0.0
      model.eval()

      for inputs, labels in valid_loader:
              val_h = tuple([each.data for each in val_h])

              inputs, labels = inputs.to(device), labels.to(device)

              output, val_h = model(inputs, val_h)
              val_loss = criterion(output.squeeze(), labels.float())
              
              val_losses.append(val_loss.item())
              
              accuracy = acc(output,labels)
              val_acc += accuracy


              
      epoch_train_loss = np.mean(train_losses)
      epoch_val_loss = np.mean(val_losses)
      
      epoch_train_acc = train_acc/len(train_loader.dataset)
      epoch_val_acc = val_acc/len(valid_loader.dataset)
      
      epoch_tr_loss.append(epoch_train_loss)
      epoch_vl_loss.append(epoch_val_loss)
      
      epoch_tr_acc.append(epoch_train_acc)
      epoch_vl_acc.append(epoch_val_acc)
      
      print(f'Epoch {epoch+1}') 
      print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss} ')
      print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
      if epoch_val_loss <= valid_loss_min:
          torch.save(model.state_dict(), path)
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
          valid_loss_min = epoch_val_loss
      print(50*'==')
  return model
training(batch_size,train_loader,valid_loader)

Epoch 1
train_loss : 0.5506239477978196 val_loss : 0.45393866300582886 
train_accuracy : 71.9298245614035 val_accuracy : 77.42382271468145
Validation loss decreased (inf --> 0.453939).  Saving model ...
Epoch 2
train_loss : 0.3843205996262011 val_loss : 0.36231583995478495 
train_accuracy : 82.40997229916897 val_accuracy : 81.16343490304709
Validation loss decreased (0.453939 --> 0.362316).  Saving model ...
Epoch 3
train_loss : 0.2706322820149651 val_loss : 0.3528894292456763 
train_accuracy : 88.47337642351492 val_accuracy : 83.37950138504155
Validation loss decreased (0.362316 --> 0.352889).  Saving model ...
Epoch 4
train_loss : 0.1828302895837976 val_loss : 0.3645839233483587 
train_accuracy : 92.19759926131117 val_accuracy : 83.10249307479224
Epoch 5
train_loss : 0.1151748696529819 val_loss : 0.40578421843903406 
train_accuracy : 95.42936288088643 val_accuracy : 83.65650969529086
Epoch 6
train_loss : 0.06820989741633336 val_loss : 0.45488001831940245 
train_accuracy : 97.16835949

LSTM(
  (embedding): Embedding(13182, 64)
  (lstm): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (lin): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [ ]:
def predict_text(text):
        word_seq = np.array([vocab[preprocess(word)] for word in text.split() 
                         if preprocess(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)
        pad =  torch.from_numpy(padding(word_seq,500))
        inputs = pad.to(device)
        batch_size = 1
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, h)
        return(output)

In [ ]:
def get_classification_metric(testy, probs):
    from sklearn.metrics import precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(testy, probs)
    # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return fscore[ix]

In [ ]:
def test_acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label).item()

In [ ]:
index = 0
pre_acc=0.0
pred=[]
test_x=X_test
test_y=y_test
print(len(test_x),len(test_y))
for i in test_x:
  prediction = predict_text(i)
  status = "negative" if prediction > 0.5 else "positive"
  prediction= 0 if status=="positive" else 1
  pred.append(prediction)
  
  #print(test_y[index],pre_acc,prediction,status)
  index+=1
  

f1=f1_score(test_y,pred)

722 722


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
print("="*50)
print("acc",accuracy_score(y_test, pred)*100)
print("F1 Score:",f1*100)
print("="*50)
print("roc",roc_auc_score(test_y, pred)*100)

acc 85.87257617728532
F1 Score: 76.49769585253456
roc 83.19934297577223
